# Amazon SageMaker - Bring Your Own Model 
## Chainer 編

ここでは [Chainer](https://chainer.org) のサンプルコードをAmazon SageMaker 上で実行するための移行手順について説明します。SageMaker Python SDK で Chainer を使うための説明は [SDK のドキュメント](https://sagemaker.readthedocs.io/en/stable/using_chainer.html) にも多くの情報があります。

注: 
また、ここでは移行手順の紹介のためトレーニングスクリプトは最小限の書き換えとしています。

## 1. トレーニングスクリプトの書き換え

### 書き換えが必要な理由
Amazon SageMaker では、オブジェクトストレージ Amazon S3 をデータ保管に利用します。例えば、S3 上の学習データを指定すると、自動的に  Amazon SageMaker の学習用インスタンスにデータがダウンロードされ、トレーニングスクリプトが実行されます。トレーニングスクリプトを実行した後に、指定したディレクトリにモデルを保存すると、自動的にモデルがS3にアップロードされます。

トレーニングスクリプトを SageMaker に持ち込む場合は、以下の点を修正する必要があります。
- 学習用インスタンスにダウンロードされた学習データのロード
- 学習が完了したときのモデルの保存

これらの修正は、トレーニングスクリプトを任意の環境に持ち込む際の修正と変わらないでしょう。例えば、自身のPCに持ち込む場合も、`/home/user/data` のようなディレクトリからデータを読み込んで、`/home/user/model` にモデルを保存したいと考えるかもしれません。同様のことを SageMaker で行う必要があります。

### 書き換える前に保存先を決める

このハンズオンでは、S3からダウンロードする学習データ・バリデーションデータと、S3にアップロードするモデルは、それぞれ以下のように学習用インスタンスに保存することにします。`/opt/ml/input/data/train/`といったパスに設定することは奇異に感じられるかもしれませんが、これらは環境変数から読み込んで使用することが可能なパスで、コーディングをシンプルにすることができます。[1-1. 環境変数の取得](#env)で読み込み方法を説明します。

#### 学習データ
- 画像: `/opt/ml/input/data/train/image.npy`
- ラベル: `/opt/ml/input/data/train/label.npy`

#### バリデーションデータ
- 画像: `/opt/ml/input/data/test/image.npy`
- ラベル: `/opt/ml/input/data/test/label.npy`

#### モデル
`/opt/ml/model` 以下にシンボルやパラメータを保存する

### 書き換える箇所
まず [サンプルのソースコード](https://github.com/chainer/chainer/blob/v5/examples/mnist/train_mnist.py) を以下のコマンドでダウンロードします。

In [ ]:
!wget https://raw.githubusercontent.com/chainer/chainer/v5/examples/mnist/train_mnist.py

ダウンロードされた `train_mnist.py` をファイルブラウザから見つけて開いて下さい (JupyterLab の場合は左右にファイルを並べると作業しやすいです)。あるいはお好きなエディターをお使い頂いても結構です。この`train_mnist.py`は、トレーニングスクリプト内で以下の関数を呼び出し、S3以外からデータをダウンロードしています。

```python
train, test = chainer.datasets.get_mnist()
```

こういった方法も可能ですが、今回はS3から学習データをダウンロードして、前述したように`/opt/ml/input/data/train/`といったパスから読み出して使います。書き換える点は主に3点です:

1. 環境変数の取得  
    SageMaker では、学習データやモデルの保存先はデフォルトで指定されたパスがあり、これらを環境変数から読み込んで使用することが可能です。環境変数を読み込むことで、学習データの位置をトレーニングスクリプト内にハードコーディングする必要がありません。もちろんパスの変更は可能で、API経由で渡すこともできます。
    
1. 学習データのロード  
    環境変数を取得して学習データの保存先がわかれば、その保存先から学習データをロードするようにコードを書き換えましょう。

1. 学習済みモデルの保存形式と出力先の変更  
    SageMaker では TensorFlow の SavedModel 形式をサポートし、TensorFlow Serving によってデプロイします。もとの`cnn_mnist.py`では、checkpointが保存されるのみでデプロイに十分な情報がありません。SavedModel 形式で保存されるようにコードを追加します。その際、モデルの保存先を正しく指定する必要があります。学習が完了すると学習用インスタンスは削除されますので、保存先を指定のディレクトリに変更して、モデルがS3にアップロードされるようにします。

### <a name="env"></a>1-1. 環境変数の取得

Amazon SageMaker の Script Mode では、トレーニングに用いるコードが実行時に Python スクリプトとして実行されます。その際、データ・モデルの入出力は [こちら](https://sagemaker.readthedocs.io/en/stable/using_chainer.html) に記述があるよう `SM_CHANNEL_XXXX` や `SM_MODEL_DIR` という環境変数を参照する必要があります。そのため、`argparse.ArgumentParser` で渡された環境変数と、スクリプト実行時のハイパーパラメータを取得します。

![データのやりとり](../img/sagemaker-data-model.png)

`SM_CHANNEL_TRAIN`, `SM_CHANNEL_TEST`, `SM_MODEL_DIR` の環境変数の値を取得するよう、以下をトレーニングスクリプトに追加します。ここでは、main関数の中でハイパーパラメータが参照されているので、main関数の中で　`parser = argparse.ArgumentParser()` の次の行にこれらを追加します。下記を実行すると、`SM_OUTPUT_DATA_DIR'` , `SM_CHANNEL_TRAIN` ,`SM_CHANNEL_TEST`, `SM_MODEL_DIR` の３つの環境変数を、`args.train`、`args.test`、`args.model_dir` に格納します。

```
    parser.add_argument('--output-data-dir', type=str, default=os.environ['SM_OUTPUT_DATA_DIR'])
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--train', type=str, default=os.environ['SM_CHANNEL_TRAIN'])
    parser.add_argument('--test', type=str, default=os.environ['SM_CHANNEL_TEST'])
```

これらの値は、create-training-jobのAPIを実行する際に (SageMaker Python SDK で estimator を呼び出す際に) 指定した hyperparameters の値に置き換えることができます。例えば、hyperparameters に `train`、`test`、`model-dir`が指定されていれば、環境変数の値は hyperparameters の値で上書きされます。

`train_mnist.py`は、`if __name__ == "__main__":`から`main():`を実行します。これで学習データ・バリデーションデータの保存先を取得することができました。次にこれらのファイルを実際に読み込む処理を実装します。

### 1-2. 学習データのロード

元のコードでは `chainer.datasets.get_mnist` を利用してダウンロード・読み込みを行っています。具体的には、`main()`のなかにある以下の1行です。今回はS3からデータをダウンロードするため、これらのコードは不要です。**ここで削除しましょう**。
```
    train, test = chainer.datasets.get_mnist()
```

代わりにS3からダウンロードしたデータを読み込みコードを実装しましょう。環境変数から取得した `train_dir`や`test_dir` にデータを保存したディレクトリへのパスが保存され、それぞれ `/opt/ml/input/data/train`, `/opt/ml/input/data/test` となります。詳細は [ドキュメント](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo-running-container.html#your-algorithms-training-algo-running-container-trainingdata) をご覧下さい。

今回は npz のファイルを読むようにコードを書き換えれば良いので、以下のようなコードを追記します。パスが `train_dir`, `test_dir` に保存されていることをうまく利用しましょう。もとの npy のデータタイプは uint8 ですが、TensorFlow のレイヤーが対応する float32 や int32 に変換したり、画像の値を 0 から 1 の範囲内になるようにします。
```
  import os
    train_data = np.load(os.path.join(args.train, 'train.npz'))['images']
    train_labels = np.load(os.path.join(args.train, 'train.npz'))['labels']

    test_data = np.load(os.path.join(args.test, 'test.npz'))['images']
    test_labels = np.load(os.path.join(args.test, 'test.npz'))['labels']

    train = chainer.datasets.TupleDataset(train_data, train_labels)
    test = chainer.datasets.TupleDataset(test_data, test_labels)
```

#### 確認

ここまでの修正で main(unused_argv) の冒頭の実装が以下の様になっていることを確認しましょう。

```
def main():
    parser = argparse.ArgumentParser(description='Chainer example: MNIST')

    # Data, model, and output directories
    parser.add_argument('--output-data-dir', type=str, default=os.environ['SM_OUTPUT_DATA_DIR'])
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--train', type=str, default=os.environ['SM_CHANNEL_TRAIN'])
    parser.add_argument('--test', type=str, default=os.environ['SM_CHANNEL_TEST'])
    parser.add_argument('--training-steps', type=int, default=20000)
```

### 1-3. 学習済みモデルの保存形式と出力先の変更

Chainerでは `model_fn()` を実装することによって、モデルに対して入力を定義することができます。また、冒頭で`import numpy as np`を追加し、以下の関数を、トレーニングスクリプトの末尾に追加します。ここでは、***

```
def model_fn(model_dir):
    chainer.config.train = False
    model = chainer.links.Classifier(MLP(1000, 10))
    chainer.serializers.load_npz(os.path.join(model_dir, 'model.npz'), model)
    return model.predictor
```

ただし、このままではこの関数は実行されず、SavedModel 形式のモデルは保存されません。main関数の最後でこの関数を呼びつつ、`export_savedmodel`でモデルが保存されるようにします。main 関数の最後は以下のような実装になります。

```
    # Run the training
    trainer.run()
    # ... train `model`, then save it to `model_dir` as file 'model.npz'
    chainer.serializers.save_npz(os.path.join(args.model_dir, 'model.npz'), model)
```
ここで、`save_npz` で出力先の `model_dir` を指定することで、学習終了後にモデルが S3 にアップロードされるようにします。

## 2. Notebook 上でのデータ準備

トレーニングスクリプトの書き換えは終了しました。　学習を始める前に、予め Amazon S3 にデータを準備しておく必要があります。この Notebook を使ってその作業をします。

In [ ]:
import os
import numpy as np
import boto3
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

機械学習に利用する手書き数字データセットの MNIST を利用します。`keras.datasets`を利用してデータセットをダウンロードし、それぞれ npy 形式で保存します。dataset のテストデータ `(X_test, y_test)` はさらにバリデーションデータとテストデータに分割します。学習データ `X_train, y_train` とバリデーションデータ `X_valid, y_valid` のみを学習に利用するため、これらを npy 形式でまずは保存します。

これを Amazon S3 にアップロードします。

In [ ]:
import os
import shutil
import numpy as np

train_images = np.array([data[0] for data in train])
train_labels = np.array([data[1] for data in train])
test_images = np.array([data[0] for data in test])
test_labels = np.array([data[1] for data in test])

try:
    os.makedirs('/tmp/data/train')
    os.makedirs('/tmp/data/test')

    np.savez('/tmp/data/train/train.npz', images=train_images, labels=train_labels)
    np.savez('/tmp/data/test/test.npz', images=test_images, labels=test_labels)

    train_input = sagemaker_session.upload_data(
        path=os.path.join('/tmp/data', 'train'),
        key_prefix='notebook/chainer/mnist')
    test_input = sagemaker_session.upload_data(
        path=os.path.join('/tmp/data', 'test'),
        key_prefix='notebook/chainer/mnist')
finally:
    shutil.rmtree('/tmp/data')

## 3. Local Mode によるトレーニングとコードの検証
トレーニングジョブを始める前に、Local Mode を使って、この Notebook インスタンス上でコンテナを実行してコードをデバッグしましょう。
`from sagemaker.tensorflow import TensorFlow` で読み込んだ SageMaker Python SDK の TensorFlow Estimator を作ります。

ここでは、学習に利用するインスタンス数 `train_instance_count` や  インスタンスタイプ `train_instance_type` を指定します。Local modeの場合は、`train_instance_type = "local"` と指定します。

デバッグなので多くの学習ステップを回す必要はありません。`training-steps` を hyperparameters で渡すことができるようになりましたので、`hyperparameters = {"training-steps": 100}` だけ実行するようにします。

In [ ]:
from sagemaker.chainer.estimator import Chainer

instance_type = "local"

mnist_estimator = Chainer(entry_point='train_mnist.py',
                             role=role,
                             train_instance_count=1,
                             train_instance_type=instance_type,
                             framework_version='5.0.0',
#                             py_version='py3',
#                             hyperparameters = {"training-steps": 100}
                         )


`estimator.fit` によりトレーニングを開始しますが、ここで指定する「チャネル」によって、環境変数名 `SM_CHANNEL_XXXX` が決定されます。この例の場合、`'train', 'test'` を指定しているので、`SM_CHANNEL_TRAIN`, `SM_CHANNEL_TEST` となります。

In [ ]:
mnist_estimator.fit({'train': train_input, 'test': test_input})

`cnn_mnist.py` の中で書き換えを忘れた部分があったら、ここでエラーとなる場合があります。Local Mode ではクイックにデバッグができるので、正しく実行できるよう試行錯誤しましょう。

 `===== Job Complete =====`
と表示されれば成功です。

### 学習済みモデルの確認

Amazon S3 に保存されたモデルは普通にダウンロードして使うこともできます。保存先は `estimator.model_data` で確認できます。

In [ ]:
mnist_estimator.model_data

AWS CLI を使ってノートブックインスタンス上にモデルをダウンロードして試しに推論します。

このノートブックと同じディレクトリに tar.gz の形式でモデルをダウンロードして展開します。展開後のディレクトリ名は数字の羅列 (Unix time) になります。あとでモデルを読み込むため、正規表現を利用して、このフォルダ名を `model_dir` に保存します。

In [ ]:
!aws s3 cp $mnist_estimator.model_data ./
!tar -zxvf ./model.tar.gz 

In [ ]:
import re    
dir_list = os.listdir(".")
pattern = "[0-9]+"
for d in dir_list:
    if re.match(pattern,d):
        model_dir = d
print("model is downloaded to ./{}".format(model_dir))

テストデータセットからランダムに10枚選んでテストを行います。先ほど保存した`model_dir`からモデルをロードして、テストデータを入力します。ローカルモードでは学習を少ししか実行しなかったため、ほとんど正しい予測はできていないと思います。

In [ ]:
test_size = 10

select_idx = np.random.choice(np.arange(y_test.shape[0]), test_size)
test_sample = X_test[select_idx].reshape([test_size,28,28]) * 1./255
    
with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.loader.load(sess, ["serve"], model_dir)
    graph = tf.get_default_graph()
    predict = sess.run('softmax_tensor:0',feed_dict={'input_layer:0': test_sample})
    for i, pred in enumerate(predict):
        print("Predict: {}, Ground Truth: {}".format(np.argmax(pred), y_test[select_idx][i]))

## 4. トレーニングジョブの発行

推論されていればコードのデバッグは完了です。次に、Amazon SageMaker のトレーニングジョブとしてトレーニングします。データ・モデルの入出力は変わらず S3 なので、`train_instance_type` に `ml.` で始まる SageMaker のインスタンスを指定するだけで実行できます。(リストは[こちら](https://aws.amazon.com/sagemaker/pricing/instance-types/))

In [ ]:
instance_type = "ml.m4.xlarge"

mnist_estimator = TensorFlow(entry_point='cnn_mnist.py',
                             role=role,
                             train_instance_count=1,
                             train_instance_type=instance_type,
                             framework_version='1.13',
                             py_version='py3',
                             hyperparameters = {"training-steps": 2500})

In [ ]:
mnist_estimator.fit({'train': train_data, 'test': valid_data})

----
```
Billable seconds: <time>
```
と出力されればトレーニング終了です。これが実際にトレーニングインスタンスが課金される時間となります。

## 5. 推論エンドポイントのデプロイ

`mnist_estimator.deploy` で、トレーニングしたモデルを推論エンドポイントとしてデプロイすることができます。これには数分かかります。(`----!` と表示されればデプロイ完了です。)

In [ ]:
predictor = mnist_estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

In [ ]:
test_size = 10
select_idx = np.random.choice(np.arange(y_test.shape[0]), test_size)
test_sample = X_test[select_idx].reshape([test_size,28,28]) * 1./255

for i in range(test_size):
    result = predictor.predict(test_sample[i])
    prediction = result['predictions'][0]['classes']
    print("Predict: {}, Ground Truth: {}".format(prediction, y_test[select_idx][i]))

推論エンドポイントは立てっぱなしにしているとお金がかかるので、確認が終わったら忘れないうちに削除してください。

In [ ]:
predictor.delete_endpoint()

## 6. まとめ

TensorFlow を使った Amazon SageMaker への移行手順について紹介しました。普段お使いのモデルでも同様の手順で移行が可能ですのでぜひ試してみてください。